<a href="https://colab.research.google.com/github/RatnamRitesh21/Ritesh_MLHW/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn import impute, tree
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# reading the Test and Train data
# We use read.csv to load the data
data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data = test_data.drop(["Name", "Ticket","Cabin"], axis=1)
print(data)

# Step1 : Preprocess the data
# Here we are using the SimpleImputer function from sklearn to fill the empty columns in Age and Embarked columns
# We are using most_frequent strategy to fill these columns with the frequent values
imputer = SimpleImputer(strategy='most_frequent')
data['Age'] = imputer.fit_transform(data['Age'].values.reshape(-1, 1)).ravel()
print(imputer.fit_transform(data['Age'].values.reshape(-1, 1)).shape)
print(imputer.fit_transform(data['Embarked'].values.reshape(-1, 1)).shape)
data['Embarked'] = imputer.fit_transform(data['Embarked'].values.reshape(-1, 1)).ravel()


# We are using the LabelEncoder function to encode the Sex and Embarked Columns
# this encoder will assign the numerical values to the columns,such as 0,1,2
encoder = LabelEncoder()
data['Sex'] = encoder.fit_transform(data['Sex'])
data['Embarked'] = encoder.fit_transform(data['Embarked'])
data = data.drop(["Name","Ticket","Cabin"], axis=1)
print(data)

# Step 2: Feature Selection
# Here we are first selecting few useful features from the columns thaare suitable to get the acurate DT
# We are now performing feature selection, from these feature using the feature_importance function
# this function selects the features whose importance is >0.04
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']
X = data.drop(["Survived"], axis=1)
Y = data['Survived']
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X,Y)
feature_importances = random_forest.feature_importances_
selected_features = [features[i] for i in range(len(features)) if feature_importances[i] > 0.04]
X_train= X[selected_features]

# Build the decision tree using the inbuilt function, the max depth parameter determines the size of the tree
DT = tree.DecisionTreeClassifier(max_depth=3)
# Train the Decision Tree classifier using the training data
DT.fit(X_train, Y)
# plotting the decision tree
tree.plot_tree(DT,class_names = ['Survived','Not Survived'])
plt.show()


# In the below code we use the same logic to preprocess the test data
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']
imputer = SimpleImputer(strategy='most_frequent')
test_data['Age'] = imputer.fit_transform(test_data['Age'].values.reshape(-1, 1)).ravel()
print(imputer.fit_transform(test_data['Age'].values.reshape(-1, 1)).shape)
print(imputer.fit_transform(test_data['Embarked'].values.reshape(-1, 1)).shape)
test_data['Embarked'] = imputer.fit_transform(test_data['Embarked'].values.reshape(-1, 1)).ravel()

# We are using the LabelEncoder function to encode the Sex and Embarked Columns
# this encoder will assign the numerical values to the columns,such as 0,1,2
encoder = LabelEncoder()
test_data['Sex'] = encoder.fit_transform(test_data['Sex'])
test_data['Embarked'] = encoder.fit_transform(test_data['Embarked'])
print(test_data)
test1_data=test_data[selected_features]

# Here we are predicting the values using the test data
# we are using the predict function on the fine tuned decision tree
classified_val = DT.predict(test1_data)
classified_data = pd.DataFrame({"PassengerId":test_data.PassengerId,"Survived":classified_val})
print(classified_data)

# Step 4 : 5 fold cross validation on DT
# Performing CrossValidation and finding the accuracy of the DT classifier
# Here we are using the cross_val_score function to perform the validation
cv_scores = cross_val_score(DT, X, Y, cv=5, scoring='accuracy')
averageDT_accuracy = cv_scores.mean()
print("Average Decision Tree Accuracy:", averageDT_accuracy)

# Step 5: Cross valication on Randon forest
# Performing cross validation of Randomforest Classifier, and caliculating the accuracy
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X,Y)
cv_scores1 = cross_val_score(random_forest, X, Y, cv=5, scoring='accuracy')
averageRF_accuracy = cv_scores1.mean()
print("Average Random Forest Accuracy:", averageRF_accuracy)


# Question 4: 6)Which algorithm is better, Decision Tree or Random Forest?
# Ans) Clearly, in my opinion, random forests outperform decision trees
# because they amalgamate multiple decision trees to generate independent trees
# that exhibit greater diversity and increased efficiency compared to individual decision trees at each branching point.


# Question 4: 7)What are your observations and conclusions from the algorithm comparison and analysis?
# Upon implementing both algorithms,
# I observed that the random classifier achieved higher accuracy than the decision tree.
#  Furthermore, the performance evaluation, which involved cross-validation in this instance,
# provided a more comprehensive understanding of the accuracy levels for both models, namely Decision Trees and Random Forest.




# Question 5 : Bagging Classifier

bagging_classifier = BaggingClassifier(tree.DecisionTreeClassifier(random_state=42), n_estimators=200, random_state=42)
bagging_classifier.fit(X,Y)
# Apply cross-validation to estimate the average classification accuracy
cv_scores_bagging = cross_val_score(bagging_classifier, X, Y, cv=5, scoring='accuracy')
average_accuracy_bagging = cv_scores_bagging.mean()
print("Average Bagging Accuracy:", average_accuracy_bagging)


#Question 6: Adaboost classifier

adaboost_classifier = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=42), n_estimators=200, random_state=42,learning_rate=0.5)
adaboost_classifier.fit(X,Y)
# Apply cross-validation to estimate the average classification accuracy
cv_scores_adaboost = cross_val_score(adaboost_classifier, X, Y, cv=5, scoring='accuracy')
average_accuracy_adaboost = cv_scores_adaboost.mean()
print("Average AdaBoost Accuracy:", average_accuracy_adaboost)

